In [114]:
import pandas as pd
import numpy as np
import hashlib
import csv
import os
from googletrans import Translator
from fuzzywuzzy import fuzz, process

#Encuentra el delimitador de un archivo
def _delimitador (path):
    if path[-4:] == ".txt":
        delimiter = ","
    else:
        data = open(path, "r", encoding="latin_1").read()
        delimiter = csv.Sniffer().sniff(data).delimiter
    return delimiter  

#Encuentra archivos de directorio y sub directorio 
def _archivos(path):
    archivos = []
    rutas = []
    with os.scandir(path) as ficheros:
        for i in ficheros:
            directorio, extencion = os.path.splitext(i)
            if extencion != "":
                ruta = directorio + extencion
                arch = os.path.basename(ruta)
                archivos.append(arch)
                rutas.append(ruta)
                #print(ruta)
            else:
                _archivos(directorio)

    return rutas, archivos


def archivos (path):
    #cantidad = Path().iterdir()
    archs = os.walk(path)
    for i,e,archivo in archs:
        return i,archivo

        
def _traducirColumnas (df):
    translator = Translator()
    for i in df.columns:
        if i == "Entity":
            df.rename(columns={i:"País"}, inplace=True)
        else:
            df.rename(columns={i:translator.translate(i, dest="es").text.title()}, inplace=True)
    return df


def _crearhash(df):
    for i in df.columns:
        if i == "País" or i == "Combustible" or i == "Año":
            nom = "Id_" + i
            if df[i].dtype == 'int64':
                df[i] = df[i].astype(str)
                df[nom] = df[i].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
                df[i] = df[i].astype(int)
            else:
                nom = "Id_" + i
                df[nom] = df[i].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
    return df


def _fuzz(df, colum):
    correcto = ['Biomass',
                'Coal',
                'Cogeneration',
                'Gas',
                'Geothermal',
                'Hydro',
                'Nuclear',
                'Oil',
                'Other',
                'Petcoke',
                'Solar',
                'Storage',
                'Waste',
                'Wave and Tidal',
                'Wind',
                'all energy types',
                'natural gas',
                'petroleum and other liquids',
                'nuclear',
                'renewables and other']
    df[colum] = df[colum].apply(lambda x: process.extractOne(x,correcto)[0])


def _limpieza_energy (df):

    df = _traducirColumnas(df)

    otro = pd.read_csv("/home/dio-pc/Escritorio/Lab-grup/archivosPrueba/owid-energy-consumption-source.csv", delimiter=",", encoding="utf-8")
    otro = otro[["year","country","population","gdp"]]
    df = pd.merge(df, otro, left_on=["Año","País"], right_on=["year","country"], how="left")
    df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False), "Población"] = df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False)]["population"]
    df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False), "Pib"] = df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False)]["gdp"]
    df.drop(columns=["country", "year", "population","gdp"], inplace=True)

    df.dropna(subset=["Emisión De Co2"], inplace=True)

    for i in df.columns:
        if i == "Tipo_Energia":
            df.rename(columns={i:"Combustible"}, inplace=True)
        elif "Energ" in i:
            nom = i + " " + "TWh"
            df[nom] = df[i] / 3.412e+12

    df = _fuzz(df, "Combustible")
    df = _crearhash(df)


def _limpieza_general (df):
    for i in df.columns:
        if "Entity" in i or "Country_long" in i:
            df.rename(columns={i:"País"})
        else:
            continue
    df = _traducirColumnas(df)
    df = _crearhash(df)

    return df


def _dataFrame (root):
    rutas, archivos = _archivos(root)
    for i in rutas:
        for e in archivos:
            if e in i:
                if e == "energyco2.csv":
                    deli = _delimitador(i)
                    nombre = "Normal_" + e 
                    df = pd.read_csv(i, low_memory=False)
                    df = _limpieza_energy(df)
                    salida = "/home/dio-pc/Escritorio/Lab-grup/Prueba/" + nombre
                    df.to_csv(salida, index=False)                   
                else:
                    deli = _delimitador(i)
                    nombre = "Normal_" + e 
                    df = pd.read_csv(i, low_memory=False)
                    df = _limpieza_general(df)
                    salida = "/home/dio-pc/Escritorio/Lab-grup/Prueba/" + nombre
                    df.to_csv(salida, index=False)
        


In [100]:
archivos("/home/dio-pc/Escritorio/emissionsmap/documentation/data")

('/home/dio-pc/Escritorio/emissionsmap/documentation/data',
 ['owid-energy-consumption-source.csv',
  'energy.csv',
  'energyco2.csv',
  'global_power_plant_database.csv',
  'sources.csv',
  'electricity-prod-source-stacked.csv',
  'primary-sub-energy-source.csv',
  'owid-energy-datadiccionary.csv',
  'historical_emissions.csv',
  'testeando2.ipynb',
  'CO2 emissions per capita per country.csv',
  'testeando_csv.ipynb',
  'greenhouse-gas-emissions-by-gas.csv',
  'sub-energy-fossil-renewables-nuclear.csv'])

In [115]:
_dataFrame("/home/dio-pc/Escritorio/Lab-grup/archivosPrueba")

AttributeError: 'NoneType' object has no attribute 'columns'

In [ ]:
def _tabla_hecho():
    pass
    #sub_ener = pd.read_csv("/home/dio-pc/Escritorio/emissionsmap/documentation/data/sub-energy-fossil-renewables-nuclear.csv")
    #final = {
    #"Año":año,
    #"País":pais,
    #"Combustible":combustible,
    #"Continente":lista
    #}